In [1]:
import numpy as np
from scipy.optimize import fsolve
from numba import njit

In [2]:
%matplotlib inline
import matplotlib as mpl

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'font.family': 'serif', 'mathtext.fontset': 'dejavuserif',
                 'font.size': 12, 'text.latex.preamble': r"\usepackage{amsmath}",
                 'xtick.major.pad': 2, 'ytick.major.pad': 2, 'xtick.major.size': 6, 'ytick.major.size': 6,
                 'xtick.minor.size': 3, 'ytick.minor.size': 3, 'axes.linewidth': 2, 'axes.labelpad': 1})

def format_axis(ax: mpl.axes._axes.Axes) -> None:
    ax.minorticks_on(); ax.grid(visible=True, which='major', linestyle=':')
    ax.tick_params(axis='both', which='both', direction='out')
    ax.xaxis.set_ticks_position('both')
    ax.yaxis.set_ticks_position('both')
    ax.patch.set_alpha(0.0)

# Exact Solution

In [3]:
def Keplers_equation(M, e):
    func = lambda x: x - e*np.sin(x) - M
    return fsolve(func, M)[0]

vKeplers_equation = np.vectorize(Keplers_equation)

In [4]:
a_test = 16.0  # semi-major axis
e_test = 63/64  # eccentricity
b_test = a_test * (1.0 - e_test ** 2.0) ** 0.5  # semi-minor axis
c_test = a_test * e_test  # focal distance axis
(a_test, e_test, b_test, c_test)

(16.0, 0.984375, 2.817356917396161, 15.75)

In [5]:
M_exact = np.linspace(0, 432/64, 432*64+1, dtype=float)  # mean anomaly
E_exact = vKeplers_equation(M_exact, e_test)  # eccentric anomaly

In [6]:
rxy_exact = np.empty((2, 432*64+1), dtype=float)  # exact position
rxy_exact[0] = (np.cos(E_exact) - e_test) * a_test
rxy_exact[1] = np.sin(E_exact) * b_test

In [7]:
EM_test = -1 / (a_test*2.0)  # mechanic energy
r0x_test = a_test - c_test  # initial position
v0y_test = (1/a_test * (1+e_test)/(1-e_test)) ** 0.5  # initial velocity
Lz_test = (a_test * (1-e_test**2.0)) ** 0.5  # angular momentum
(EM_test, r0x_test, v0y_test, Lz_test)

(-0.03125, 0.25, 2.817356917396161, 0.7043392293490403)

In [8]:
@njit  # conservation correction
def CC_rxy_to_vxy(rxy, vxy, EM, Lz):
    vxy[0] = -rxy[1] * Lz
    vxy[1] = rxy[0] * Lz

    r2 = np.sum(np.square(rxy))
    sqrt_Deltar = max(2.0 * (r2**0.5 + EM*r2) - Lz**2.0, 0.0) ** 0.5
    vxy += np.sign(rxy[1]) * rxy * sqrt_Deltar

    vxy /= r2

In [9]:
vxy_exact = np.empty((2, 432*64+1), dtype=float)  # exact velocity
for i in range(432*64+1):
    CC_rxy_to_vxy(rxy_exact[:, i], vxy_exact[:, i], EM_test, Lz_test)

In [10]:
@njit  # mechanic energy
def EM_from_rxy_vxy(rxy, vxy):
    # r = np.sqrt(np.sum(np.square(rxy)))
    # v2 = np.sum(np.square(vxy))
    return -1.0/np.sqrt(np.sum(np.square(rxy))) + np.sum(np.square(vxy))/2.0

@njit  # angular momentum
def Lz_from_rxy_vxy(rxy, vxy):
    return rxy[0]*vxy[1] - rxy[1]*vxy[0]

In [11]:
exact_plot = False  # 4/28/2024, plot everything versus mean anomaly

if exact_plot:
    EM_recover = np.empty((432*64+1,), dtype=float)  # ``recovered'' mechanic energy
    Lz_recover = np.empty((432*64+1,), dtype=float)  # ``recovered'' angular momentum
    for i in range(432*64+1):
        EM_recover[i] = EM_from_rxy_vxy(rxy_exact[:, i], vxy_exact[:, i])
        Lz_recover[i] = Lz_from_rxy_vxy(rxy_exact[:, i], vxy_exact[:, i])

In [12]:
if exact_plot:
    fig, axs = plt.subplots(7, figsize=(12.8, 12.8), sharex='col')

    for ax, data, label in zip(axs, [E_exact, *rxy_exact, *vxy_exact, EM_recover - EM_test, Lz_recover - Lz_test],
                               ['Eccentric anomaly', 'Position ($x$ comp)', 'Position ($y$ comp)',
                                'Velocity ($x$ comp)', 'Velocity ($y$ comp)', '$\Delta$ Mechanic energy', '$\Delta$ Angular momentum']):
        ax.scatter(M_exact, data, c='tab:blue', s=0.1, rasterized=True)
        ax.set_ylabel(label)
        format_axis(ax)

    axs[-1].set_xlabel('Mean anomaly')
    axs[-1].set_xlim((-0.05, M_exact[-1]+0.05))
    fig.tight_layout()
    fig.subplots_adjust(hspace=0.1)
    plt.savefig('eccentric_exact.pdf', dpi=128)